<font size="+3"><strong>Machine Learning: Unsupervised Learning</strong></font>

Machine Learning falls into two classes, **supervised** learning and **unsupervised** learning. In supervised learning, we're trying to learn a predictive relationship between **features** of our data and some sort of  **target**. In unsupervised learning, we want to find trends in our features without using any target. 

A human example of supervised learning would be borrowing books from a library on mathematics and geography. By reading different books belonging to each topic, we learn what symbols, images, and words are associated with math, and which are associated with geography. A similar unsupervised task would be to borrow many books without knowing their subject. We can see some books contain similar images (maps) and some books contain similar symbols (e.g. the Greek letters $\Sigma$ and $\pi$). We say the books containing maps are similar and that they are different from the books containing Greek letters. Crucially, _we do not know what the books are about, only that they are similar or different_.

# Clustering

Clustering is a branch of unsupervised machine learning where the goal is to identify groups or clusters in your data set without the use of labels. Clustering should not be considered the same as classification. We are not trying make predictions on observations from a set of pre-defined classes. In clustering, you are identifying a set of similar data points and calling the resulting set a cluster.

Let's consider an example of clustering. You may have a data set characterizing your customers like demographic information and personal preferences. A supervised machine learning task would be to predict which class a person belongs to: customers who will purchase your product and customers who won't. In contrast, an _unsupervised_ machine learning task would be to identify several groups or types of customers. With these groups identified, you can analyze the groups and build profiles describing the groups. For example, one group tends to include people from the ages 20 to 25 who like the outdoors. With these profiles, you can pass that information and analysis to the marketing team to create different advertisements to best attract each group.

## k-means Clustering

The k-means algorithms seeks to find $K$ clusters within a data set. The clusters are chosen to reduce the distance of the data points within each cluster. The objective function is

$$ \min_{C_k} \sum_{k} \sum_{X_j \in  C_k} \| X_j  - \mu_k \|^2. $$

where $\mu_k$ is defined as the **centroid** of a cluster. Each cluster has a unique $\mu_k$, which equals to the _mean_ of each feature/components of all points assigned to the cluster. We use the following equation to calculate $\mu_k$:

$$ \mu_k = \frac{1}{|C_k|} \sum_{X_j \in C_k} X_j, $$

here $|C_k|$ is the number of points in cluster $k$. The training algorithm for k-means is iterative. First, we assign $k$ random starting locations as each cluster's centroid, then we:

1. Assign each point to a cluster based on which cluster centroid it's closest to
1. Calculate a new centroid for each cluster based its the datapoints
1. Repeat the above steps until convergence is met

To see how the algorithm works in practice, let's quickly go through this example below:

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


def wrangle(filepath):
    df = pd.read_csv(filepath)
    mask = df["TURNFEAR"] == 1
    df = df[mask]
    return df


# Read Data into DataFrame
df = wrangle("data/SCFP2019-textbook.csv.gz")
print(df.shape)
df.head()

In the following example, we'll use `"INCOME"` and `"HOUSES"` to demonstrate the k-means clustering algorithm. First, we select the two features from our DataFrame as training set.

In [ ]:
X = df[["INCOME", "HOUSES"]]
print(X.shape)
X.head()

Next, we apply the k-means clustering algorithm from `sklearn`. We can choose the number of clusters to by defining `n_clusters`. In this example, we will show the results with 3 clusters. Note the algorithm starts with randomly assigned initial centroid positions, so defining a `random_state` will make sure the randomly assigned positions stay the same for repetitive runs. 

In [ ]:
model = KMeans(n_clusters=3, random_state=42)
# Fit model to data
model.fit(X)

After fitting the data, the model will assign each data point a `label`, indicating which cluster this data point belongs to.

In [ ]:
labels = model.labels_
labels[:10]

To visualize the clustering results, we can quickly draw a scatter plot showing the two features. We can use different colors for different clusters.

In [ ]:
sns.scatterplot(
    x=df["INCOME"] / 1e6, y=df["HOUSES"] / 1e6, hue=model.labels_, palette="deep"
)

plt.xlabel("Household Income [$1M]")
plt.ylabel("Home Value [$1M]")
plt.title("Home Value vs. Household Income");

From the scatter plot, we can see the k-means algorithm divides data points mostly based on the `HOUSE` feature. The lower home value data points are assigned to cluster 0, higher home value data points are assigned to cluster 1, while cluster 2 shows data points with home value in the middle.

As mentioned in describing the k-means algorithm, **centroid** plays a very important role in deciding the clustering results. We can extract the final locations of centroid from each cluster, and plot them in the scatter plot.

In [ ]:
# Extract centroid
centroids = model.cluster_centers_
centroids

In [ ]:
sns.scatterplot(
    x=df["INCOME"] / 1e6, y=df["HOUSES"] / 1e6, hue=model.labels_, palette="deep"
)
plt.scatter(
    centroids[:, 0] / 1e6, centroids[:, 1] / 1e6, color="gray", marker="*", s=150
)

plt.xlabel("Household Income [$1M]")
plt.ylabel("Home Value [$1M]")
plt.title("Home Value vs. Household Income");

## Clustering Metrics

To see whether our clustering algorithm performs well, we need more than a scatter plot. The two common metrics we used are **inertia** and **silhouette score**. These metrics will also be helpful in determining the number of clusters to use.

### Inertia

 **Inertia** is the within-cluster sum of square distance, which is used in k-means algorithm's objective function. Mathematically, inertia is equal to

$$ \sum_{k} \sum_{X_j \in  C_k} \| X_j  - \mu_k \|^2, $$

where $\mu_k$ is the centroid of cluster $k$ and $C_k$ is the set of points assigned to cluster $k$. Basically, the inertia is the sum of the distance of each point to the centroid or center of its assigned cluster. A lower inertia means the points assigned to the clusters are closer to the centroid.

We can extract `inertia` from the previous model using the code below:

In [ ]:
inertia = model.inertia_
print("Inertia (3 clusters):", inertia)

### Silhouette Score

**Silhouette Coefficient** is a measure of how dense and separated are the clusters. The silhouette coefficient is a property assigned to each data point. It's equal to

$$ \frac{b - a}{\max(a, b)}, $$

where $a$ is the distance between a point and centroid of its assigned cluster; $b$  is the distance between the point and the centroid of the nearest neighboring cluster (i.e. the closest cluster the point is not assigned to).

The silhouette coefficient ranges from -1 to 1. If a point is really close to the centroid of its assigned cluster, then $a \ll b$ and the silhouette coefficient will be approximately equal to 1. If the reverse is true, $a \gg b$, then the coefficient will be -1. If the point could have been assigned to either cluster, its coefficient will be 0.

Higher silhouette coefficient means higher density and highly separated clusters. This is because we want to have lower $a$ (close to assigned cluster's centroid) and higher $b$ (far away from unassigned cluster's centroid). A lower $a$ value combined with higher $b$ value will produce a higher silhouette score.

We can extract calculate the silhouette score using the code below:

In [ ]:
ss = silhouette_score(X, model.labels_)
print("Silhouette Score (3 clusters):", ss)

## Optimizing the Number of Clusters

We can choose the optimal number of clusters by examining how number of cluster affect inertia and silhouette score. Let's check the following plots showing how inertia and silhouette scores change with respect to number of clusters ranging from 2 to 20.

In [ ]:
n_clusters = range(2, 20)
inertia_errors = []
silhouette_scores = []

for n in n_clusters:
    model = KMeans(n_clusters=n, random_state=42)
    model.fit(X)
    inertia_errors.append(model.inertia_)
    silhouette_scores.append(silhouette_score(X, model.labels_))

Now we have saved the metrics, we can plot them.

In [ ]:
plt.plot(n_clusters, inertia_errors)
plt.xlabel("Number of Clusters")
plt.ylabel("Inertia")
plt.title("k-means Model: Inertia vs Number of Clusters");

Note that inertia will decrease whenever the number of cluster increases. In fact, inertia will go to zero if the number of clusters equals number of data points, because each data point would be its own cluster. But that wouldn't make any practical sense, so we're not looking for the minimum point on the graph. Instead,we want the point where increasing numbers of clusters will not decrease inertia that much anymore. We usually refer to the point that indicating this change of inertia decreasing speed as the **elbow point**. In the example here, the elbow point is at 4-5.

We can also plot the silhouette scores:

In [ ]:
plt.plot(n_clusters, silhouette_scores)
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Score")
plt.title("k-means Model: Silhouette Score vs Number of Clusters");

From the graph, we can see the silhouette score dropped a lot from 2 to 4, and from 5 to 6. Note that a higher silhouette score means denser and more clearly separated clusters, which is what we want. Combining both the inertia plot and the silhouette score plot, we can see the optimal number of cluster should be at 5. 

<font size="+1">Practice</font> 

Use `ASSET` and `INCOME` from the same data set `"SCFP2019-textbook.csv.gz"`, and:

1. Use k-means to assign the data points to 3 clusters.
1. Create a scatter plot showing the resulting clusters and their centroids.
1. Calculate the inertia and the silhouette score.

In [ ]:
# Get data
def wrangle(filepath):
    df = pd.read_csv(filepath)
    mask = df["TURNFEAR"] == 1
    df = df[mask]
    return df


# Read Data into DataFrame
df = ...

# Select Features
X = ...

# Define Model
model = ...

# Fit model to data


In [ ]:
# Get centroids
centroids = ...

# Plot "ASSET" vs "HOUSES" with hue=label


plt.xlabel("Household Income [$1M]")
plt.ylabel("Home Value [$1M]")
plt.title("Home Value vs. ASSET");

In [ ]:
inertia = ...
print("Inertia (3 clusters):", inertia)

In [ ]:
ss = ...
print("Silhouette Score (3 clusters):", ss)

# Principal Component Analysis

Principal component analysis (PCA) is a dimension reduction technique that takes a data set characterized by a set of possibly correlated features and generates a new set of features that are uncorrelated. It is used as a dimension reduction technique because the new set of uncorrelated features are chosen to be efficient in terms of capturing the variance in the data set.

Let's examine a case where we have a data set of only two dimensions. In practice, PCA is rarely used when the dimension of the data set is already low. However, it is easier to illustrate the method when we have two or three dimensions.

In [ ]:
import numpy as np

np.random.seed(0)
x1 = np.linspace(0, 1, 500)
x2 = 2 * x1 + 1 + 0.2 * np.random.randn(500)
X = np.vstack((x1, x2)).T

plt.scatter(*X.T, alpha=0.25)
plt.plot(x1, 2 * x1 + 1, "--k", linewidth=2)
plt.xlabel("$x_1$")
plt.ylabel("$x_2$");

The data plotted is characterized by two dimensions, but most of the variation does not occur in either of the two dimensions. Most of the points "follow" along the direction plotted in the dashed line. The variables $x_1$ and $x_2$ are highly correlated; as $x_1$ increases, in general, so does $x_2$ and vice versa.

Instead of using the original two features, $x_1$ and $x_2$, perhaps we can use a different set of features, $\xi_1$ and $\xi_2$. The first chosen feature $\xi_1$ should be aligned in the direction of greatest variation while the second will be _orthogonal_ to the first. The new axes/dimensions are referred to as _principal components_. Let's visualize the data set but using the principal components $\xi_1$ and $\xi_2$ rather than the original features.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
Xt = pca.fit_transform(X)

xi_1_max, xi_2_max = Xt.max(axis=0)
xi_1_min, xi_2_min = Xt.min(axis=0)

plt.hlines(0, xi_1_min, xi_1_max, linestyles="--")
plt.vlines(0, xi_2_min, xi_2_max, linestyles="--")

plt.scatter(*Xt.T, alpha=0.25)
plt.xlim([-1.75, 1.75])
plt.ylim([-1.75, 1.75])
plt.xlabel("$\\xi _1$")
plt.ylabel("$\\xi _2$");

In the figure, we can clearly observe that $\xi_1$ is the dimension with the largest variation. In the PCA algorithm, $\xi_1$ is chosen to capture as much of the variation as possible, with $\xi_2$ picking up the rest of remaining variation. Now, if we want to use one dimension to describe our data, we would keep $\xi_1$ and drop $\xi_2$, ensuring we keep as much of the information in our data set using just one dimension. Further, notice how the new dimensions are not correlated. As we move from lower to higher values of $\xi_1$, $\xi_2$ does not predictability increase or decrease.

## PCA in `scikit-learn`

In `scikit-learn`, dimension reduction algorithms are transformers. The choice of having these algorithms as transformers makes sense since they apply a transformation on the data set. Let's illustrate the syntax for the PCA algorithm in `scikit-learn`. For most of these algorithms, the data needs to be centered and scaled to work properly. `PCA` automatically centers the data but **does not** scale it. `StandardScaler` is often used for preprocessing the data prior to applying PCA.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Define trained data
df = wrangle("data/SCFP2019-textbook.csv.gz")
X = df[["INCOME", "HOUSES", "DEBT", "NETWORTH", "NFIN", "ASSET"]]

# Scaling data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the model
pca = PCA(n_components=2)

# Fit and transform data
Xt = pca.fit_transform(X_scaled)

print("Number of dimension before reduction:", X_scaled.shape[-1])
print("Number of dimension after reduction:", Xt.shape[-1])

---
Copyright © 2022 WorldQuant University. This
content is licensed solely for personal use. Redistribution or
publication of this material is strictly prohibited.
